In [ ]:
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
!pip install adjustText
from shapely.geometry import Point
from adjustText import adjust_text

In [ ]:
def quarter_start(year: int, q: int) -> datetime:
    if not 1 <= q <= 4:
        raise ValueError("Quarter must be within [1, 2, 3, 4]")

    month = [1, 4, 7, 10]
    return datetime(year, month[q - 1], 1)


def get_tile_url(service_type: str, year: int, q: int) -> str:
    dt = quarter_start(year, q)

    base_url = "https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance"
    url = f"{base_url}/type%3D{service_type}/year%3D{dt:%Y}/quarter%3D{q}/{dt:%Y-%m-%d}_performance_{service_type}_tiles.zip"
    return url

In [ ]:
tile_url = get_tile_url("mobile", 2020, 2)
tile_url

'https://ookla-open-data.s3-us-west-2.amazonaws.com/shapefiles/performance/type%3Dmobile/year%3D2020/quarter%3D2/2020-04-01_performance_mobile_tiles.zip'

In [ ]:
tiles = gp.read_file(tile_url)
tiles.head()
# tiles.to_csv('ooklatiles_q2-202002.csv', index=False)

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,0022133222330012,13040,19446,67,2,1,"POLYGON ((-160.03784 70.63995, -160.03235 70.6..."
1,0022133222330013,11702,20146,68,1,1,"POLYGON ((-160.03235 70.63995, -160.02686 70.6..."
2,0022133222330023,6124,20987,85,1,1,"POLYGON ((-160.04333 70.63631, -160.03784 70.6..."
3,0022133222330030,1687,1262,100,2,1,"POLYGON ((-160.03784 70.63813, -160.03235 70.6..."
4,0022133222330201,8261,9320,90,11,1,"POLYGON ((-160.04333 70.63448, -160.03784 70.6..."


In [ ]:
tiles.shape



(4095011, 7)

In [ ]:
print(tiles.columns)

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry'],
      dtype='object')


In [ ]:
sample_quadkey = tiles['quadkey'].iloc[0]
print("Length of sample quadkey:", len(sample_quadkey))

Length of sample quadkey: 16


In [ ]:
!pip install mercantile

import mercantile

# Define Turkey's bounding box
TURKEY_BBOX = {
    'min_lon': 25.0,
    'min_lat': 35.0,
    'max_lon': 45.0,
    'max_lat': 43.0
}

# Generate tiles for a specific zoom level (e.g., 12 for regional granularity)
zoom_level = 16
tiles = list(mercantile.tiles(
    west=TURKEY_BBOX['min_lon'],
    south=TURKEY_BBOX['min_lat'],
    east=TURKEY_BBOX['max_lon'],
    north=TURKEY_BBOX['max_lat'],
    zooms=zoom_level
))

# Convert tiles to quadkeys
quadkeys = [mercantile.quadkey(tile) for tile in tiles]

print(f"Generated {len(quadkeys)} quadkeys for Turkey at zoom level {zoom_level}")
print(quadkeys)


Generated 6837798 quadkeys for Turkey at zoom level 16
Buffered data was truncated after reaching the output size limit.

In [ ]:
!pip install mercantile

import mercantile
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors

# Define Turkey's bounding box
TURKEY_BBOX = {
    'min_lon': 25.0,
    'min_lat': 35.0,
    'max_lon': 45.0,
    'max_lat': 43.0
}

# Generate tiles for a specific zoom level (e.g., 16 for regional granularity)
zoom_level = 16
tiles = list(mercantile.tiles(
    west=TURKEY_BBOX['min_lon'],
    south=TURKEY_BBOX['min_lat'],
    east=TURKEY_BBOX['max_lon'],
    north=TURKEY_BBOX['max_lat'],
    zooms=zoom_level
))

# Convert tiles to quadkeys
quadkeys = [mercantile.quadkey(tile) for tile in tiles]
print(f"Generated {len(quadkeys)} quadkeys for Turkey at zoom level {zoom_level}")

# Prepare to plot
fig, ax = plt.subplots(figsize=(12, 8))

# Set the color map to visualize different quadkeys
# Color map (can be modified depending on how many quadkeys you have)
cmap = plt.cm.get_cmap("tab20", len(quadkeys))  # 'tab20' is a qualitative colormap

# Loop through each tile and plot its bounding box
for idx, tile in enumerate(tiles):
    # Get the bounding box for the current tile
    min_lon, min_lat, max_lon, max_lat = mercantile.bounds(tile)

    # Create a rectangle for the bounding box (this represents the quadkey area)
    rect = patches.Rectangle(
        (min_lon, min_lat), max_lon - min_lon, max_lat - min_lat,
        linewidth=1, edgecolor=cmap(idx), facecolor=cmap(idx), alpha=0.5
    )
    ax.add_patch(rect)

# Set the axis labels
plt.title(f"Quadkey Visualization for Turkey (Zoom Level {zoom_level})")
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Set limits for the plot based on Turkey's bounding box
ax.set_xlim(TURKEY_BBOX['min_lon'], TURKEY_BBOX['max_lon'])
ax.set_ylim(TURKEY_BBOX['min_lat'], TURKEY_BBOX['max_lat'])

# Show the plot
plt.show()


Generated 6837798 quadkeys for Turkey at zoom level 16


<ipython-input-11-3524ad349a15>:34: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap("tab20", len(quadkeys))  # 'tab20' is a qualitative colormap


KeyboardInterrupt: 